In [1]:
import re
import ipywidgets as widgets
from IPython.display import display, HTML, Image
import graphviz

# Token types for lexical analysis
class TokenType:
    MATH_OPERATOR = "MATH_OPERATOR"
    NUMERIC_VALUE = "NUMERIC_VALUE"
    PARENTHESIS = "PARENTHESIS"
    SYMBOL = "SYMBOL"

# Token structure for lexical analysis
class Token:
    def _init_(self, type, value):
        self.type = type
        self.value = value

# DFA structure for pattern matching
class DFA:
    def _init_(self, pattern):
        self.pattern = pattern
        self.transition_table = {}

        # Build transition table
        for state in range(len(pattern) + 1):
            for char in set(pattern):
                next_state = min(len(pattern), state + 1)
                while next_state > 0 and pattern[:next_state] != pattern[state-next_state+1:state] + char:
                    next_state -= 1
                self.transition_table[(state, char)] = next_state

    def match(self, text):
        state = 0
        matches = []
        for i, char in enumerate(text):
            state = self.transition_table.get((state, char), 0)
            if state == len(self.pattern):
                matches.append(i - len(self.pattern) + 1)
        return matches

    def generate_diagram(self):
        dot = graphviz.Digraph(format='png')
        for state in range(len(self.pattern) + 1):
            dot.node(str(state), shape='circle', style='filled', fillcolor='lightblue2' if state == len(self.pattern) else 'white')
        for state, char in self.transition_table:
            dot.edge(str(state), str(self.transition_table[(state, char)]), label=char)
        return dot

    def get_transition_table_html(self):
        rows = []
        for state in range(len(self.pattern) + 1):
            row = [f"<td>{state}</td>"]
            for char in sorted(set(self.pattern)):
                next_state = self.transition_table.get((state, char), 0)
                row.append(f"<td>{next_state}</td>")
            rows.append("<tr>" + "".join(row) + "</tr>")
        header = "<tr><th>State</th>"
        for char in sorted(set(self.pattern)):
            header += f"<th>{char}</th>"
        header += "</tr>"
        return f"<table border='1' cellspacing='0' cellpadding='5'>{header}{''.join(rows)}</table>"

# Tokenize the input expression
def tokenize_expression(expression):
    tokens = []
    number_buffer = ""

    for c in expression:
        if c.isspace():
            continue
        elif c.isdigit() or c == '.':
            number_buffer += c
        else:
            if number_buffer:
                tokens.append(Token(TokenType.NUMERIC_VALUE, number_buffer))
                number_buffer = ""

            if c in {'+', '-', '*', '/', '^'}:
                tokens.append(Token(TokenType.MATH_OPERATOR, c))
            elif c in {'(', ')'}:
                tokens.append(Token(TokenType.PARENTHESIS, c))
            else:
                tokens.append(Token(TokenType.SYMBOL, c))

    if number_buffer:
        tokens.append(Token(TokenType.NUMERIC_VALUE, number_buffer))

    return tokens

# Function to validate mathematical expression
def validate_expression(expression):
    try:
        # Attempt to evaluate the expression
        eval(expression)
        return True
    except:
        return False

# Function to perform lexical analysis on input expression
def perform_lexical_analysis(expression):
    if not validate_expression(expression):
        result_text.value = "<b style='color:red;'>Error: Invalid mathematical expression</b>"
        # Disable DFA diagram and transition table buttons
        display_dfa_button.disabled = True
        display_transition_button.disabled = True
        return

    # If expression is valid, enable DFA diagram and transition table buttons
    display_dfa_button.disabled = False
    display_transition_button.disabled = False

    tokens = tokenize_expression(expression)

    result = f"<b>Mathematical Expression:</b> {expression}<br><br>"
    result += "<b>Tokens:</b><br>"
    for token in tokens:
        result += f"<b>{token.type}:</b> {token.value}<br>"
    result_text.value = result

    # Evaluate the expression and display the result
    try:
        result_value = eval(expression)
        result_text.value += f"<br><br><b>Result:</b> {result_value}"
    except Exception as e:
        result_text.value += f"<br><br><b style='color:red;'>Error: {str(e)}</b>"

# Function to display DFA diagram
def display_dfa(button):
    pattern = expression_text.value.strip()
    dfa = DFA(pattern)
    display(Image(data=dfa.generate_diagram().pipe(format='png')))

# Function to display transition table
def display_transition_table(button):
    pattern = expression_text.value.strip()
    dfa = DFA(pattern)
    display(HTML(dfa.get_transition_table_html()))

# GUI
style = """
<style>
    .custom-widget {
        background-color: #000000;
        padding: 10px;
        border-radius: 5px;
        margin-bottom: 10px;
        color: #ffffff;
    }
    .custom-button {
        background-color: #1E90FF;
        border: none;
        color: #ffffff;
        padding: 10px 20px;
        text-align: center;
        text-decoration: none;
        display: inline-block;
        font-size: 14px;
        margin: 4px 2px;
        cursor: pointer;
        border-radius: 5px;
        vertical-align: middle; /* Align text vertically */
        line-height: 20px; /* Adjust line height */
    }
</style>
"""

display(HTML(style))

expression_text = widgets.Textarea(
    value='',
    placeholder='Enter a mathematical expression',
    description='',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

analyze_button = widgets.Button(description="Analyze", style={'description_width': 'initial'})
analyze_button.on_click(lambda b: perform_lexical_analysis(expression_text.value))

display_dfa_button = widgets.Button(description="Display DFA Diagram", style={'description_width': 'initial'})
display_dfa_button.on_click(display_dfa)

display_transition_button = widgets.Button(description="Display Transition Table", style={'description_width': 'initial'})
display_transition_button.on_click(display_transition_table)

result_text = widgets.HTML(
    value='',
    placeholder='Results will be displayed here',
    description='',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

expression_text.add_class("custom-widget")
analyze_button.add_class("custom-button")
display_dfa_button.add_class("custom-button")
display_transition_button.add_class("custom-button")
result_text.add_class("custom-widget")

widgets.VBox([expression_text, analyze_button, display_dfa_button, display_transition_button, result_text])